In [1]:
import kagglehub
# kagglehub.login()
kagglehub.dataset_download('hlgsagar1234567/vr-go')
kagglehub.dataset_download("adityaav80/validatiioon")

'/kaggle/input/validatiioon'

In [35]:
import pandas as pd
df = pd.read_csv("/kaggle/input/validatiioon/val_red.csv")
print(len(df))
df.head(5)

50050


,image_id,image_path,question,answer
0,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the main color of the phone case?,Black
1,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the object depicted on the phone case?,UFO
2,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,Where is the camera lens positioned on the pho...,Top
3,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,How many stars are visible around the UFO on t...,Seven
4,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the phone case covering?,Phone


In [18]:
!pip install qwen-vl-utils hf_xet dagshub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.


In [15]:
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
import torch

# Load processor & model (with trust_remote_code to pick up the custom BLIP2 code)
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", 
    trust_remote_code=True,
    use_fast=True
)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
new_model = model.to(device)

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import re
import os
import transformers
from qwen_vl_utils import process_vision_info
import dagshub
from dagshub import get_repo_bucket_client


def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower().strip())

def create_message(image_path, question):
    return [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path,
                },
                {"type": "text", "text": question},
            ],
        }
    ]

st,en = 0,1
start_row = int(50050/20)*st+1
end_row =  int(50050/20)*en
output_csv = f"/kaggle/working/evaluation_{st}_{en}.csv"


baai = SentenceTransformer("BAAI/bge-base-en-v1.5")

results = []
image_path = None  # initialize outside loop

for i in range(start_row, min(end_row, len(df))):
    try:
        # Always set image_path from df
        if i % 10 == 0 or image_path is None:
            image_path = df.loc[i, "image_path"]
            image = Image.open(image_path).convert("RGB")
        
        question = df.loc[i, "question"]
        gt = df.loc[i, "answer"]

        prompt1 = f"{question}. Answer the question in **one word** only."
        prompt2 = f"{question}."

        messages1 = create_message(image_path, prompt1)
        messages2 = create_message(image_path, prompt2)

        inputs_short = processor.apply_chat_template(messages1, tokenize=False, add_generation_prompt=True)
        inputs_long = processor.apply_chat_template(messages2, tokenize=False, add_generation_prompt=True)

        image_inputs1, video_inputs1 = process_vision_info(messages1)
        image_inputs2, video_inputs2 = process_vision_info(messages2)

        inputs1 = processor(
            text=[inputs_short],
            images=image_inputs1,
            videos=video_inputs1,
            padding=True,
            return_tensors="pt",
        ).to("cuda")

        inputs2 = processor(
            text=[inputs_long],
            images=image_inputs2,
            videos=video_inputs2,
            padding=True,
            return_tensors="pt",
        ).to("cuda")

        with torch.no_grad():
            generated_ids1 = model.generate(**inputs1, max_new_tokens=5)
            pred_short = processor.batch_decode(
                [out[len(inp):] for inp, out in zip(inputs1.input_ids, generated_ids1)],
                skip_special_tokens=True, clean_up_tokenization_spaces=False
            )

            generated_ids2 = model.generate(**inputs2, max_new_tokens=50)
            pred_long = processor.batch_decode(
                [out[len(inp):] for inp, out in zip(inputs2.input_ids, generated_ids2)],
                skip_special_tokens=True, clean_up_tokenization_spaces=False
            )
            
            torch.cuda.empty_cache()

        pred_short_clean = clean_text(pred_short)
        pred_long_clean = clean_text(pred_long)
        gt_clean = clean_text(str(gt))

        if pred_short_clean and gt_clean:
            emb_short = baai.encode(pred_short_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True, show_progress_bar=False)
            sim_short = min(1.0, round(util.cos_sim(emb_short, emb_gt).item(), 3))
        else:
            sim_short = ""
        
        if pred_long_clean and gt_clean:
            emb_long = baai.encode(pred_long_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True, show_progress_bar=False)
            sim_long = min(1.0, round(util.cos_sim(emb_long, emb_gt).item(), 3))
        else:
            sim_long = ""

        exact_match = 1 if pred_short_clean == gt_clean and pred_short_clean else 0

        results.append({
            "index": i,
            "image_path": image_path,
            "question": question,
            "answer": gt,
            "qwen_prediction_short": pred_short_clean,
            "qwen_prediction_long": pred_long_clean,
            "baai_similarity_short": sim_short,
            "baai_similarity_long": sim_long,
            "qwen_exact_match_short": exact_match
        })

        if (i - start_row + 1) % 50 == 0:
            pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
            print(f"[SAVED] Up to row {i}")
            results = []

    except Exception as e:
        print(f"[ERROR] at row {i}: {e}")
        continue

# Final save
if results:
    pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))

    os.environ["DAGSHUB_USER_TOKEN"] = '8f9637537a95e70f478dc0a0ebc62e1970b28b5b'
    os.environ["MLFLOW_TRACKING_PASSWORD"] = '8f9637537a95e70f478dc0a0ebc62e1970b28b5b'

    bucket_client = get_repo_bucket_client("adityaav80/VR_Results", flavor="boto")

    bucket_client.upload_file(
        Filename=output_csv,
        Bucket="VR_Results",
        Key=f"results/qwen/evaluation_{st}_{en}.csv"  # Remote path inside DagsHub storage
    )
    print("[UPLOADED] to DagsHub storage.")